In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import monai
from monai.utils import first, set_determinism, ensure_tuple_rep, look_up_option, optional_import, progress_bar, set_determinism
from monai.transforms import (
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
    Transform,
    AsDiscrete,
    RandFlipd,
    RandShiftIntensityd,
    RandRotate90d,
    EnsureTyped,
    MapTransform,
    ResizeWithPadOrCropd,
    CropForeground,
    Resize,
    RandAffined
)

from monai.networks.nets import SwinUNETR, UNet,SegResNet
from monai.handlers.utils import from_engine
from monai.networks.layers import Norm, DropPath, trunc_normal_
from monai.networks.blocks import MLPBlock as Mlp
from monai.networks.blocks import PatchEmbed, UnetOutBlock, UnetrBasicBlock, UnetrUpBlock
from monai.networks import normal_init
from monai.metrics import DiceMetric, MeanIoU, HausdorffDistanceMetric
from monai.losses import DiceLoss,  DiceCELoss ,GeneralizedDiceLoss,MaskedDiceLoss,DiceFocalLoss
from monai.inferers import sliding_window_inference
from monai.data import ThreadDataLoader, CacheDataset, DataLoader, load_decathlon_datalist, Dataset, decollate_batch, set_track_meta
from monai.config import print_config
from monai.apps import download_and_extract




######from main.aa import SegResNet





import itertools
from typing import Optional, Sequence, Tuple, Type, Union

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
from torch.nn import LayerNorm
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

#from __future__ import annotations

#from collections.abc import Sequence

#import numpy as np
#import torch
#import torch.nn as nn
#import torch.nn.functional as F

#from monai.networks.blocks.segresnet_block import ResBlock, get_conv_layer, get_upsample_layer
#from monai.networks.layers.factories import Dropout
#from monai.networks.layers.utils import get_act_layer, get_norm_layer
from monai.utils import UpsampleMode
#__all__ = ["SegResNet", "SegResNetVAE"]
from monai.transforms import LoadImaged, AddChanneld, Compose, Resized

print_config()

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = "savedmodel"
#root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

set_determinism(seed=0)

data_dir = "../MONAI"
train_images1 = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr1", "*.nii.gz")))
train_images2 = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr2", "*.nii.gz")))
train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))



data_dicts = [
    {"image1": image1_name, "image2": image2_name, "label": label_name}
    for image1_name, image2_name, label_name in zip(train_images1, train_images2, train_labels)
]



#train_files, val_files = data_dicts[:-5], data_dicts[-5:]
train_files, val_files = data_dicts[:-8], data_dicts[-8:]

In [29]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image1","image2", "label"]),
        EnsureChannelFirstd(keys=["image1","image2", "label"]),
        #ScaleIntensityRanged(
            #keys=["image"], a_min=0, a_max=1,
            #b_min=0.0, b_max=1.0, clip=True,
        #),
        
        #CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image1","image2", "label"], axcodes="RAS"),
        #Resized(
            #keys=["image1", "image2", "label"],
            #spatial_size=(112,112,64),
            #mode=['bilinear','bilinear','nearest']
        #),
        Spacingd(keys=["image1","image2", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear","bilinear", "nearest")),
        RandCropByPosNegLabeld(
            keys=["image1","image2", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            #spatial_size=(112, 112, 112),
            pos=1,
            neg=1,
            num_samples=18,
            image_threshold=0,
        ),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image1","image2", "label"]),
        EnsureChannelFirstd(keys=["image1","image2", "label"]),
        #ScaleIntensityRanged(
            #keys=["image"], a_min=0, a_max=180,
            #b_min=0.0, b_max=1.0, clip=True,
        #),
        #CropForegroundd(keys=["image", "label"], source_key="image"),
        #CropForegroundd(keys=["image", "label"], source_key="label", margin=64),
        Orientationd(keys=["image1","image2", "label"], axcodes="RAS"),
        Spacingd(keys=["image1","image2", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear","bilinear", "nearest")),
    ]
)

In [ ]:
check_ds = Dataset(data=train_files, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
#print(check_data["image1"].size())
image1,image2, label = (check_data["image1"][0][0],check_data["image2"][0][0], check_data["label"][0][0])
#print(f"image1 shape: {image1.shape},image2 shape: {image2.shape}, label shape: {label.shape}")
# plot the slice [:, :, 50]
plt.figure("check", (18, 6))
plt.subplot(1, 3, 1)
plt.title("image1")
plt.imshow(image1[:, :, 30].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 2)
plt.title("image2")
plt.imshow(image2[:, :, 30].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 3)
plt.title("label")
plt.imshow(label[:, :, 30].detach().cpu())
plt.show()

In [ ]:
check_ds = Dataset(data=val_files, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
print(check_data["image1"].size())
image1,image2, label = (check_data["image1"][0][0],check_data["image2"][0][0], check_data["label"][0][0])
print(f"image1 shape: {image1.shape},image2 shape: {image2.shape}, label shape: {label.shape}")
# plot the slice [:, :, 50]
plt.figure("check", (18, 6))
plt.subplot(1, 3, 1)
plt.title("image1")
plt.imshow(image1[:, :, 42].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 2)
plt.title("image2")
plt.imshow(image2[:, :, 42].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 3)
plt.title("label")
plt.imshow(label[:, :,  42].detach().cpu())
plt.show()

In [ ]:
train_ds = CacheDataset(
    data=train_files, transform=train_transforms,
    cache_rate=1.0, num_workers=4)
# train_ds = Dataset(data=train_files, transform=train_transforms)

# use batch_size=2 to load images and use RandCropByPosNegLabeld
# to generate 2 x 4 images for network training
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4)

val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=4)
    
#val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=4)

# torch.Tensor([B, D, H, W])
# [121, 224, 224], [111, 224, 224]
#↓ (Cache)Dataset
# [5, 96, 96, 96]

In [33]:
#train_ds = Dataset(data=train_files, transform=train_transforms)
#train_loader = DataLoader(train_ds, batch_size=5)

In [ ]:
slice_map = {
    "output231_dwi.nii.gz": 80,
}

case_num = 1
#img1_name = os.path.split(val_ds[case_num]['image1'].meta["filename_or_obj"])[1]
#img2_name = os.path.split(val_ds[case_num]['image2'].meta["filename_or_obj"])[1]
img_name = os.path.split(val_ds[case_num]['image1'].meta["filename_or_obj"])[1]
img1 = val_ds[case_num]["image1"]
img2 = val_ds[case_num]["image2"]
label = val_ds[case_num]["label"]
img1_shape = img1.shape
img2_shape = img2.shape
label_shape = label.shape
print(f"image1 shape: {img1_shape},image2 shape: {img2_shape}, label shape: {label_shape}")
plt.figure("image", (18, 6))
plt.subplot(1, 3, 1)
plt.title("image1")
plt.imshow(img1[0, :, :, slice_map[img_name]].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 2)
plt.title("image2")
plt.imshow(img2[0, :, :, slice_map[img_name]].detach().cpu(), cmap="gray")
plt.subplot(1, 3, 3)
plt.title("label")
plt.imshow(label[0, :, :, slice_map[img_name]].detach().cpu())
plt.show()

In [ ]:
# standard PyTorch program style: create UNet, DiceLoss and Adam optimizer
#device = torch.device("cuda:0")
#model = UNet(
    #spatial_dims=3,
    #in_channels=1,
    #out_channels=2,
    #channels=(64, 128, 256, 512),
    #strides=(2, 2, 2),
    #num_res_units=2
#).to(device)

model = SegResNet(
    spatial_dims=3,
    init_filters=8,
    in_channels=2,
    out_channels=2,
    dropout_prob=None,
    act=('RELU', {'inplace': True}),
    norm=('GROUP', {'num_groups': 8}),
    norm_name='',
    num_groups=8,
    use_conv_final=True,
    blocks_down=(1, 2, 2, 4),
    blocks_up=(1, 1, 1),
    upsample_mode=UpsampleMode.NONTRAINABLE)


model.apply(normal_init)
# input images are scaled to [0,1] so enforce the same of generated outputs
model.conv_final.add_module("activation", torch.nn.Sigmoid())
#gen_net.model.add_module("activation", torch.nn.Sigmoid())
model = model.to(device)

#loss_function = DiceLoss(to_onehot_y=True, softmax=True)DiceFocalLoss
#loss_function = DiceLoss(include_background=True,to_onehot_y=True, softmax=True)
#loss_function =GeneralizedDiceLoss(include_background=True,to_onehot_y=True,softmax=True,)#smooth_nr=1e-05)#smooth_nr=1e-05)
#loss_function =DiceFocalLoss(to_onehot_y=True, softmax=True)
#loss_function = MaskedDiceLoss(include_background=True,to_onehot_y=True, softmax=True)
loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
#optimizer = torch.optim.Adam(model.parameters(), 3e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
#dice_metric = DiceMetric(include_background=False, reduction="mean")
dice_metric = DiceMetric(include_background=False, reduction="mean")

print (model)

In [ ]:
max_epochs = 1000 #need 600 epochs to train a promising model
val_interval = 5
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
post_pred = Compose([AsDiscrete(argmax=True, to_onehot=2)])
post_label = Compose([AsDiscrete(to_onehot=2)])

for epoch in range(max_epochs):
    print("-" * 600)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs1,inputs2, labels = (
            batch_data["image1"].to(device),
            batch_data["image2"].to(device),
            batch_data["label"].to(device),
        )
        x = torch.cat([inputs1, inputs2], dim=1)
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(
            f"{step}/{len(train_ds) // train_loader.batch_size}, "
            f"train_loss: {loss.item():.4f}")
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            for val_data in val_loader:
                val_inputs1,val_inputs2, val_labels = (
                    val_data["image1"].to(device),
                    val_data["image2"].to(device),
                    val_data["label"].to(device),
                )
                x = torch.cat([val_inputs1, val_inputs2], dim=1)
                #roi_size = (192, 192, 128)
                roi_size = (128, 128, 96)
                sw_batch_size = 2
                val_outputs = sliding_window_inference(
                    x , roi_size, sw_batch_size, model)
                val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)

            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()

            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), os.path.join(
                    root_dir, "best_metric_model.pth"))
                print("saved new best metric model")
            print(
                f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
                f"\nbest mean dice: {best_metric:.4f} "
                f"at epoch: {best_metric_epoch}"
            )

print(
    f"train completed, best_metric: {best_metric:.4f} "
    f"at epoch: {best_metric_epoch}")


In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [i + 1 for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [val_interval * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.show()

In [38]:
val_org_transforms = Compose(
    [
        LoadImaged(keys=["image1","image2", "label"]),
        EnsureChannelFirstd(keys=["image1","image2", "label"]),
        Orientationd(keys=["image1","image2"], axcodes="RAS"),
        Spacingd(keys=["image1","image2","label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear","bilinear","nearest")),
        #ScaleIntensityRanged(
            #keys=["image"], a_min=0, a_max=5036,
            #b_min=0.0, b_max=1.0, clip=True,
        #),
        #CropForegroundd(keys=["image"], source_key="image"),
    ]
)

val_org_ds = Dataset(
    data=val_files, transform=val_org_transforms)
val_org_loader = DataLoader(val_org_ds, batch_size=1, num_workers=4)

post_transforms = Compose([
    Invertd(
        keys="pred",
        transform=val_org_transforms,
        orig_keys="image",
        meta_keys="pred_meta_dict",
        orig_meta_keys="image_meta_dict",
        meta_key_postfix="meta_dict",
        nearest_interp=False,
        to_tensor=True,
        device="cpu",
    ),
    AsDiscreted(keys="pred", argmax=True, to_onehot=2),
    AsDiscreted(keys="label", to_onehot=2),
])

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(root_dir, "best_metric_model.pth")))
model = model.to(device)
model.eval()

with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs1 = val_data["image1"].to(device)
        val_inputs2 = val_data["image2"].to(device)
        
        x = torch.cat([val_inputs1, val_inputs2], dim=1).to(device)
        #roi_size = (192, 192, 128)
        #roi_size = (128, 128, 96)
        roi_size = (112, 112, 96)
        sw_batch_size = 4
        val_data["pred"] = sliding_window_inference(
            x, roi_size, sw_batch_size, model)
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        val_outputs, val_labels = from_engine(["pred", "label"])(val_data)
        # compute metric for current iteration
        dice_metric=DiceMetric(include_background=False, reduction="mean")
        jaccard_metric = MeanIoU(include_background=False, reduction="mean")
        hd = HausdorffDistanceMetric(include_background=False, reduction="mean")
        
        #print(type(val_outputs))
        #print(type(val_labels))
        #  [Tensor([],device='cuda:0')] -> Tensor([],device='cuda:0') -> Tensor([],device='cpu') ->[[Tensor([],device='cpu)]]
        dice_metric(y_pred=[val_outputs[0].cpu()], y=[val_labels[0]])
        jaccard_metric(y_pred=[val_outputs[0].cpu()], y=[val_labels[0]])
        hd(y_pred=[val_outputs[0].cpu()], y=[val_labels[0]])
        
        #dice_metric(y_pred=val_outputs, y=val_labels)
        #jaccard_metric(y_pred=val_outputs, y=val_labels)
        #hd(y_pred=val_outputs, y=val_labels)
        
        
    #metric_org_dsc = dice_metric(y_pred=val_outputs, y=val_labels)
    #metric_org_jsc = jaccard_metric(y_pred=val_outputs, y=val_labels)
    #metric_org_hd = hd(y_pred=val_outputs, y=val_labels)


    # aggregate the final mean dice result
    metric_org_dsc = dice_metric.aggregate().item()
    metric_org_jsc = jaccard_metric.aggregate().item()
    metric_org_hd = hd.aggregate().item()
    # reset the status for next validation round
    dice_metric.reset()
    jaccard_metric.reset()
    hd.reset()

print("Metric on original image spacing: ", metric_org_dsc)
print("Metric on original image spacing: ", metric_org_jsc)
print("Metric on original image spacing: ", metric_org_hd)

In [40]:
test_images1 = sorted(
    glob.glob(os.path.join(data_dir, "imagesTs1", "*.nii.gz")))
test_images2 = sorted(
    glob.glob(os.path.join(data_dir, "imagesTs2", "*.nii.gz")))
test_labels = sorted(
    glob.glob(os.path.join(data_dir, "labelsTs", "*.nii.gz")))
#test_image = [{"image": image} for image in test_images]
#test_label = [{"label": image} for image in test_labels]
test_files = [
    {"image1": image1_name,"image2": image2_name, "label": label_name}
    for image1_name,image2_name, label_name in zip(test_images1,test_images2, test_labels)
]

test_org_transforms = Compose(
    [
        LoadImaged(keys=["image1","image2", "label"]),
        EnsureChannelFirstd(keys=["image1","image2", "label"]),
        Orientationd(keys=["image1","image2"], axcodes="RAS"),
        Spacingd(keys=["image1","image2","label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear","bilinear","nearest")),
        #ScaleIntensityRanged(
            #keys=["image"], a_min=0, a_max=5036,
            #b_min=0.0, b_max=1.0, clip=True,
        #),
        #CropForegroundd(keys=["image"], source_key="image"),
    ]
)

test_org_ds = Dataset(
    data=test_files, transform=test_org_transforms)

test_org_loader = DataLoader(test_org_ds, batch_size=1, num_workers=4)


class ReverseLabels:
    def __call__(self, data):
        data["pred"] = 1 - data["pred"]  # セグメンテーション領域と背景の値を反転
        return data


post_transforms = Compose([
    Invertd(
        keys="pred",
        transform=test_org_transforms,
        orig_keys="image",
        meta_keys="pred_meta_dict",
        orig_meta_keys="image_meta_dict",
        meta_key_postfix="meta_dict",
        nearest_interp=False,
        to_tensor=True,
        device="cpu",
    ),
    #ReverseLabels(),
    AsDiscreted(keys="pred", argmax=True, to_onehot=2),
    SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="../seg/out", output_postfix="seg", resample=False),
])

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(root_dir, "best_metric_model.pth")))
model.eval()

dice_scores = []
jaccard_scores = []
hausdorff_distances = []


with torch.no_grad():
    with open("result.txt", "w") as file:
        for idx, test_data in enumerate(test_org_loader):
            test_inputs1 = test_data["image1"].to(device)
            test_inputs2 = test_data["image2"].to(device)
            x = torch.cat([test_inputs1, test_inputs2], dim=1)
            #roi_size = (192, 192, 128)
            #roi_size = (112, 112, 96)
            roi_size = (128, 128, 112)
            sw_batch_size = 1
            test_data["pred"] = sliding_window_inference(x, roi_size, sw_batch_size, model)
            test_data = [post_transforms(i) for i in decollate_batch(test_data)]
            test_outputs, test_raw_labels = from_engine(["pred", "label"])(test_data)

            plt.figure("check", (18, 6))
            plt.subplot(1, 3, 1)
            plt.imshow(test_inputs1[0, 0, :, :, 10].detach().cpu().numpy(), cmap="gray")
            
            plt.subplot(1, 3, 2)
            plt.imshow(test_inputs2[0, 0, :, :, 10].detach().cpu().numpy(), cmap="gray")

            plt.subplot(1, 3, 3)
            plt.imshow(test_outputs[0].detach().cpu()[1, :, :, 10], cmap="gray")

            dice_metric = DiceMetric(include_background=False, reduction="mean")
            jaccard_metric = MeanIoU(include_background=False, reduction="mean")
            hd = HausdorffDistanceMetric(include_background=False, reduction="mean")

            dice_metric(y_pred=[test_outputs[0].cpu()], y=test_raw_labels)
            jaccard_metric(y_pred=[test_outputs[0].cpu()], y=test_raw_labels)
            hd(y_pred=[test_outputs[0].cpu()], y=test_raw_labels)

            dice_score = dice_metric.aggregate().item()
            jaccard_score = jaccard_metric.aggregate().item()
            hausdorff_distance = hd.aggregate().item()

            dice_scores.append(dice_score)
            jaccard_scores.append(jaccard_score)
            hausdorff_distances.append(hausdorff_distance)

            file.write(f"Image {idx + 1} - Dice: {dice_score:.4f}, Jaccard: {jaccard_score:.4f}, Hausdorff Distance: {hausdorff_distance:.4f}\n")

            dice_metric.reset()
            jaccard_metric.reset()
            hd.reset()

        avg_dice = sum(dice_scores) / len(dice_scores)
        std_dice = torch.tensor(dice_scores).std()

        avg_jaccard = sum(jaccard_scores) / len(jaccard_scores)
        std_jaccard = torch.tensor(jaccard_scores).std()

        avg_hausdorff = sum(hausdorff_distances) / len(hausdorff_distances)
        std_hausdorff = torch.tensor(hausdorff_distances).std()

        file.write(f"Avg Dice: {avg_dice:.4f}, Std Dice: {std_dice:.4f}\n")
        file.write(f"Avg Jaccard: {avg_jaccard:.4f}, Std Jaccard: {std_jaccard:.4f}\n")
        file.write(f"Avg Hausdorff Distance: {avg_hausdorff:.4f}, Std Hausdorff Distance: {std_hausdorff:.4f}\n")

print("Metric on original image spacing: ", avg_dice)
print("Metric on original image spacing: ", avg_jaccard)
print("Metric on original image spacing: ", avg_hausdorff)
print("Results saved to result.txt")